In [1]:
# Basic Libraries Needed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import host, user, password
import os

# Remove the angry pink boxes
import warnings
warnings.filterwarnings("ignore")


1. Acquire `bedroomcnt`, `bathroomcnt`, `calculatedfinishedsquarefeet`, `taxvaluedollarcnt`, `yearbuilt`, `taxamount`, and `fips` from the zillow database for all 'Single Family Residential' properties.


In [2]:
def get_connection(database_name):
    '''
    This function takes in a string representing a database name for the Codeup mySQL server 
    and returns a string that can be used to open a connection to the server.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{database_name}'


In [3]:
def get_zillow_data():
    '''
    This function reads the Zillow database from the Codeup mySQL server and  returns a dataframe.
    If a local file does not exist, this function writes data to a csv file as a backup. The local file 
    ensures that data can be accessed, in the event that you cannot talk to the mySQL database. 
    '''
    # The filename will have 2017 at the end to represent that the only data being looked at is 
    # properties from the year 2017
    if os.path.isfile('zillow2017.csv'):
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow2017.csv', index_col=0)
        
    else:
        
        # Read fresh data from database into a DataFrame
        # property land use type id is limited to 'Single Family Residential' properties.
        df =  pd.read_sql(""" SELECT bedroomcnt, 
                                     bathroomcnt, 
                                     calculatedfinishedsquarefeet, 
                                     taxvaluedollarcnt, 
                                     yearbuilt, 
                                     taxamount, 
                                     fips
                              FROM properties_2017
                              WHERE propertylandusetypeid = 261;""", 
                            get_connection('zillow')
                        )
        
        # Cache data into a csv backup
        df.to_csv('zillow2017.csv')
        
    return df

In [4]:
zillow = get_zillow_data()
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.


In [5]:
zillow.shape

(2152863, 7)

In [6]:
# Average bedroom count is 3
# Average bathroom count is 2
zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [7]:
# Verbose and null_counts made true, because this dataset is a bit larger
zillow.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [8]:
# I know there are null values, but I don't know how many:
zillow.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

#### Null_Counts by Percentage  
bedroomcnt                        0.000005%  
bathroomcnt                       0.000005%  
calculatedfinishedsquarefeet      0.0039%  
taxvaluedollarcnt                 0.000229%  
yearbuilt                         0.0043%  
taxamount                         0.0021  
fips                              0%  

In [9]:
# Total count of all Nulls is 1.06% of all data. 
# I think it would be safe to drop all nulls and not have it affect my data
zillow.isnull().sum().sum()

22778

In [10]:
# These numbers don't add up, I better check again
print(zillow.shape)
zillow = zillow.dropna()
print(zillow.shape)

(2152863, 7)
(2140235, 7)


In [11]:
# Checking to make sure everything looks right
zillow.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [12]:
zillow.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2140235 non-null  float64
 1   bathroomcnt                   2140235 non-null  float64
 2   calculatedfinishedsquarefeet  2140235 non-null  float64
 3   taxvaluedollarcnt             2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


#### Notes:
- I should probably change yearbuilt to an INT since years dont have decimals

- Also, I googled fips because I didn't know what it was:  
    - Federal Information Processing System (FIPS) Codes for States and Counties. 
    - FIPS codes are numbers which uniquely identify geographic areas.

- It looks like fips are whole numbers, so I'll make that an int too!  
- Bathrooms are known to have wierd numbers like 2.5 Bath, so float makes sense  
- I'm not entirely sure a 2.5 Bedroom makes sense, but I'll leave that for now.  
- Squarefeet can definitely have a decimal  
- And tax dollars is money, so a float works just fine here


In [13]:
zillow["yearbuilt"] = zillow.yearbuilt.astype(int)
zillow["fips"] = zillow.fips.astype(int)
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     int64  
 5   taxamount                     float64
 6   fips                          int64  
dtypes: float64(5), int64(2)
memory usage: 130.6 MB


#### I can't think of anything else to clean up, so I think I'm good to combine all this into a function!

3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.


In [14]:
def wrangle_zillow (zillow):
    # Drop all data with nulls. This about 1.06% of the data, so shouldn't affect outcome
    zillow = zillow.dropna()
    
    # Change the data types of these columns to int
    zillow["yearbuilt"] = zillow.yearbuilt.astype(int)
    zillow["fips"] = zillow.fips.astype(int)
    
    return zillow